In [4]:
# LLM과 RAG 기술을 활용해 사용자 질문에 답변하는 챗봇
# PDF 파일을 불러와서 챗봇을 구축하며 이번주 배운 내용을 실습하여 복습
# 프론트 없이 백에서만 구동되는것을 확인 가능하면 OK⇒ jupyter notebook에서 확인하는 식


# openai_api_key = 
#추후 공부를 위해 참고자료 : Gemini API 발급방법
#import os
#api_key = 'Gemini api key'
#os.environ["GOOGLE_API_KEY"] = api_key 

In [1]:
!pip install langchain_openai

In [2]:
# 사용 환경을 준비
import os
from getpass import getpass
import openai

#getpass란? 사용자로부터 비밀번호를 입력받고, 그 입력이 화면에 출력되지 않도록 한다.
# getpass로 OpenAI API 키 입력 받기
api_key = getpass("Enter your OpenAI API key: ")

# 환경 변수에 API 키 설정
os.environ["OPENAI_API_KEY"] = api_key

# OpenAI API 키를 환경 변수에서 가져오기
# .getenv()는 파이썬의 os 모듈에 포함된 함수로, 환경 변수(environment variable)의 값을 가져오는 데 사용한다.
openai.api_key = os.getenv("OPENAI_API_KEY")

Enter your OpenAI API key:  ········


In [3]:
#검색 기반 생성(RAG)
#LangChain을 사용하여 질문에 답할 때 관련 문서를 검색하고, 
#해당 내용을 바탕으로 응답을 생성할 수 있다. 이를 통해 최신 정보에 기반한 답변을 생성할 수 있기에 LangChian으로 사용

#OpenAI의 GPT-4o-mini 모델을 LangChain을 통해 사용해 보자. ChatOpenAI를 이용해 초기화한다.


# openai 모델 로드하기

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

#추후 공부를 위해 참고자료 : Gemini API 모델 로드 및 모델 초기
#from langchain_google_genai import ChatGoogleGenerativeAI
#model = ChatGoogleGenerativeAI(model="gemini-pro")

# 모델 초기화, 최신 모델로
model = ChatOpenAI(model="gpt-4o-mini")

In [4]:
# 추후 공부를 위해
# 인공지능최신산업동향
    # LLM에 업데이트 되어있지 '않은' 최신 인공지능산업동향에 대해 정확히 답변할 수 있는 챗봇
    # 'RAG의 필요성'을 잘 드러내는 자료
# 초거대 언어모델 연구 동향
    # LLM에 업데이트 되어있지 '않은' 최신 인공지능 연구동향에 대해 정확히 답변할 수 있는 챗봇
    # 'LLM 요소에 익숙'해지기 좋은 자료
# 2024 세금 절약 가이드
    # LLM에 업데이트 되어있지 '않은' 한국의 세금 절약 방법에 대해 정확히 답변할 수 있음
    # 'RAG 의 필요성'을 드러내며, RAG 솔루션회사가 주로 법률, 세금 등 의 도메인에서 많이 필요로 하기에 의미있는 자료


# 첫번째 문서 로드하기
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
#import os
#file_path = os.path.join("C:", "Users", "workp", "OneDrive", "LLM&RAG", "인공지능 산업의 최신 동향.pdf")
#loader = PyPDFLoader(file_path)

file_path = "인공지능최신동향.pdf"
loader = PyPDFLoader("C:/Users/workp/OneDrive/LLM&RAG/인공지능 산업의 최신 동향.pdf")

# 페이지 별 문서 로드
docs = loader.load()

In [5]:
# 문서 청크로 나누기
# 청킹을 완수하면 청킹된 내용을 상위 50개까지 출력하고, 각 청킹방식과 parameter의 뜻을 markdown으로 정리
# i 또는 ii 사용해서 나타내기


# i. CharacterTextSplitter 
    #from langchain.text_splitter import CharacterTextSplitter
    #text_splitter = CharacterTextSplitter(
        #separator="\n\n",
        #chunk_size=100,
        #chunk_overlap=10,
        #length_function=len,
        #is_separator_regex=False,
    #)
    #splits = text_splitter.split_documents(docs)

#ii. RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)
splits = recursive_text_splitter.split_documents(docs)
splits[0:50] 

[Document(metadata={'source': 'C:/Users/workp/OneDrive/LLM&RAG/인공지능 산업의 최신 동향.pdf', 'page': 0}, page_content='2024년 11월호'),
 Document(metadata={'source': 'C:/Users/workp/OneDrive/LLM&RAG/인공지능 산업의 최신 동향.pdf', 'page': 1}, page_content='2024년 11월호'),
 Document(metadata={'source': 'C:/Users/workp/OneDrive/LLM&RAG/인공지능 산업의 최신 동향.pdf', 'page': 1}, page_content='Ⅰ. 인공지능 산업 동향 브리프 1. 정책/법제    ▹ 미국 민권위원회, 연방정부의 얼굴인식 기술 사용에 따른 민권 영향 분석························1'),
 Document(metadata={'source': 'C:/Users/workp/OneDrive/LLM&RAG/인공지능 산업의 최신 동향.pdf', 'page': 1}, page_content='▹ 미국 백악관 예산관리국, 정부의 책임 있는 AI 조달을 위한 지침 발표·····························2   ▹ 유로폴, 법 집행에서 AI의 이점과'),
 Document(metadata={'source': 'C:/Users/workp/OneDrive/LLM&RAG/인공지능 산업의 최신 동향.pdf', 'page': 1}, page_content='AI의 이점과 과제를 다룬 보고서 발간··············································3   ▹ OECD, 공공 부문의 AI 도입을 위한 G7'),
 Document(metadata={'source': 'C:/Users/workp/OneDrive/LLM&RAG/인공지능 산업의 최신 동향.pdf', 'page': 1}, page_content='도입을 위한 G7 툴

In [6]:
# 벡터 임베딩 생성
# OpenAI 모델의 경우 OpenAIEmbeddings , Gemini 모델의 경우 GoogleGenerativeAIEmbeddings를 이용해 텍스트를 벡터로 변환할 벡터 임베딩을 생성

#추후 공부를 위한 : Gemini
#GoogleGenerativeAIEmbeddings로 벡터 임베딩 생성
#from langchain_google_genai import GoogleGenerativeAIEmbeddings
#Gemini 임베딩 모델 초기
#embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001") # gemini의 임베딩 모델

# OpenAI의 OpenAIEmbeddings을 이용해서 텍스트를 벡터로 변환할 벡터 임베딩 생성
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [7]:
!pip install faiss-cpu

In [8]:
#로컬 저장소도 따로 해야하나요?

# 벡터 스토어 생성
# 앞서 만든 벡터 임베딩과 청크된 문서를 활용하여 FAISS 벡터 스토어를 생성
import faiss
from langchain_community.vectorstores import FAISS


vector_store = FAISS.from_documents(documents=splits, embedding=embeddings)

In [10]:
# RAG 체인에서 사용할 수 있도록 FAISS를 retriever로 변환
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [11]:
# LangChain의 모델과 프롬프트를 연결하여 RAG 체인을 구성
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output
# 문서 리스트를 텍스트로 변환하는 단계 추가
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        # context의 각 문서를 문자열로 결합
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}

# RAG 체인에서 각 단계마다 DebugPassThrough 추가
rag_chain_debug = {
    "context": retriever,                    # 컨텍스트를 가져오는 retriever
    "question": DebugPassThrough()        # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
}  | DebugPassThrough() | ContextToText()|   contextual_prompt | model


In [ ]:
# 챗봇 구동 확인
# 질문에 응답하는 챗봇을 구동하여 질문해보기
# 같은 질문을 일반 chat gpt 혹은 Gemini에 질문해보고 답변을 비교해보고, 왜 RAG이 필요한지 간단히 markdown으로 서술

# 무한 루프(동작 확인)
while True: 
    print("========================")
    query = input("질문을 입력하세요 (종료하려면 'exit' 입력): ")
    
    # 사용자가 'exit'을 입력하면 루프 종료
    if query.lower() == 'exit':
        print("프로그램을 종료합니다.")
        break  # 루프 종료
        
    # query를 전달하여 응답 받기
    response = rag_chain_debug.invoke(query) 
    print("Final Response:")
    print(response.content)


질문을 입력하세요 (종료하려면 'exit' 입력):  2024년 9월 19일날 정보를 알려줘


Debug Output: 2024년 9월 19일날 정보를 알려줘
Debug Output: {'context': [Document(metadata={'source': 'C:/Users/workp/OneDrive/LLM&RAG/인공지능 산업의 최신 동향.pdf', 'page': 0}, page_content='2024년 11월호')], 'question': '2024년 9월 19일날 정보를 알려줘'}
Final Response:
죄송하지만, 2024년 9월 19일에 대한 구체적인 정보는 제공할 수 없습니다.


질문을 입력하세요 (종료하려면 'exit' 입력):  인공지능이란?


Debug Output: 인공지능이란?
Debug Output: {'context': [Document(metadata={'source': 'C:/Users/workp/OneDrive/LLM&RAG/인공지능 산업의 최신 동향.pdf', 'page': 23}, page_content='-신경정보처리시스템재단은 인공지능과 머신러닝 분야의 연구 성과 교환을 촉진하는 것을 목적으로 하는 비영리 법인으로 매년 학제간 학술대회(NeurIPS)를 주최-이번 제38회')], 'question': '인공지능이란?'}
Final Response:
인공지능은 컴퓨터 시스템이 인간과 유사한 방식으로 학습, 추론, 문제 해결, 이해 및 언어 처리 등의 지능적인 작업을 수행할 수 있도록 하는 기술 및 이론을 의미합니다.


질문을 입력하세요 (종료하려면 'exit' 입력):  인공지능을 영어로 어떻게 표현하나요?


Debug Output: 인공지능을 영어로 어떻게 표현하나요?
Debug Output: {'context': [Document(metadata={'source': 'C:/Users/workp/OneDrive/LLM&RAG/인공지능 산업의 최신 동향.pdf', 'page': 22}, page_content='대체할 가능성이 제한적∙일례로 생성AI는 디지털 기술 비중이 큰 소프트웨어 개발 직종의 구인 공고에서 통상 제시되는 직무 기술의 71%에 대하여 인간을 대체할 가능성이 “보통”')], 'question': '인공지능을 영어로 어떻게 표현하나요?'}
Final Response:
인공지능은 영어로 "artificial intelligence"라고 표현합니다.


In [ ]:
#일단 공부 후 이 방식에서 수정해서 완성하기
#시간되면 도전과제도 해보기
#제출은 19일까지
#평가 기준 확인해보기

In [ ]:
#2024년 9월 19일 정보 물어본 이유 :  Document(metadata={'source': 'C:/Users/workp/OneDrive/LLM&RAG/인공지능 산업의 최신 동향.pdf', 'page': 3}, page_content='2024년 9월 19일 연방정부의 얼굴인식 기술 사용이 민권에 미치는 영향을 분석한 보고서를 발간∙AI 기술의 일종인 얼굴인식 기술은 연방정부와 법 집행기관에서 빠르게 도입되고'),
# 인공지능이란?: 인공지능은 컴퓨터 시스템이 인간과 유사한 방식으로 학습, 추론, 문제 해결, 이해 및 언어 처리 등의 지능적인 작업을 수행할 수 있도록 하는 기술 및 이론을 의미합니다.
# 인공지능을 영어로 어떻게 표현하나요?: 인공지능은 영어로 "artificial intelligence"라고 표현합니다.

#왜 RAG이 필요한지 간단히 markdown으로 서술
#RAG 모델은 자체 데이터를 기반으로 지식 저장소를 만들고, 이로부터 AI가 상황에 맞는 답변을 적시에 제공할 수 있도록 지원하기에, 동적으로 활도함으로써 보다 유연한 답변을 할 수 있기에 RAG 모델이 필요하다고 생각합니다.